![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

# Training and Reusing Assertion Status Models

In [0]:
import os
import json
import string
import numpy as np
import pandas as pd


import sparknlp
import sparknlp_jsl
from sparknlp.base import *
from sparknlp.util import *
from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp.pretrained import ResourceDownloader

from pyspark.sql import functions as F
from pyspark.ml import Pipeline, PipelineModel

pd.set_option('max_colwidth', 100)
pd.set_option('display.max_columns', 100)  
pd.set_option('display.expand_frame_repr', False)


print('sparknlp_jsl.version : ',sparknlp_jsl.version())

spark

sparknlp_jsl.version : 3.0.1
Out[1]:

SparkSession - hive 
 
 
 SparkContext 

 Spark UI 

 
 Version 
 v3.0.1 
 Master 
 spark://10.164.250.91:7077 
 AppName 
 Databricks Shell

# Clinical Assertion Model (with pretrained models)

## Pretrained `assertion_dl` model

In [0]:
# Annotator that transforms a text column from dataframe into an Annotation ready for NLP

from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
  .setInputCol("text")\
  .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line

sentenceDetector = SentenceDetector()\
  .setInputCols(["document"])\
  .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP

tokenizer = Tokenizer()\
  .setInputCols(["sentence"])\
  .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
  .setInputCols(["sentence", "token"])\
  .setOutputCol("embeddings")

# NER model trained on i2b2 (sampled from MIMIC) dataset
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = NerConverter() \
  .setInputCols(["sentence", "token", "ner"]) \
  .setOutputCol("ner_chunk")

# Assertion model trained on i2b2 (sampled from MIMIC) dataset
# coming from sparknlp_jsl.annotator !!
clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
    
nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    clinical_ner,
    ner_converter,
    clinical_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
model = nlpPipeline.fit(empty_data)

embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][OK!]
ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][ / ][OK!]

In [0]:
# we also have a LogReg based Assertion Model.
'''
clinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")
'''

Out[4]: '\nclinical_assertion_ml = AssertionLogRegModel.pretrained("assertion_ml", "en", "clinical/models") .setInputCols(["sentence", "ner_chunk", "embeddings"]) .setOutputCol("assertion")\n'

In [0]:
import pandas as pd

text = 'Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain'

print (text)

light_model = LightPipeline(model)

light_result = light_model.fullAnnotate(text)[0]

chunks=[]
entities=[]
status=[]

for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
    
    chunks.append(n.result)
    entities.append(n.metadata['entity']) 
    status.append(m.result)
        
df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

df

Patient has a headache for the last 2 weeks and appears anxious when she walks fast. No alopecia noted. She denies pain
Out[12]:

,chunks,entities,assertion
0,a headache,PROBLEM,present
1,anxious,PROBLEM,conditional
2,alopecia,PROBLEM,absent
3,pain,PROBLEM,absent


In [0]:
from sparknlp_display import AssertionVisualizer

visualizer = AssertionVisualizer()

vis = visualizer.display(light_result, 'ner_chunk', 'assertion', return_html=True)
#visualizer.set_label_colors({'TREATMENT':'#008080', 'PROBLEM':'#800080'})


displayHTML(vis)

Patient has a headache PROBLEM present for the last 2 weeks and appears anxious PROBLEM conditional when she walks fast. No alopecia PROBLEM absent noted. She denies pain PROBLEM absent

In [0]:
! wget 	https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv


--2021-04-29 20:31:07-- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/pubmed/pubmed_sample_text_small.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.217.110.22
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.217.110.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9363435 (8.9M) [text/csv]
Saving to: ‘pubmed_sample_text_small.csv’

 pubmed_sa 0%[ ] 0 --.-KB/s  pubmed_sam 1%[ ] 91.60K 427KB/s  pubmed_samp 7%[> ] 652.60K 1.48MB/s  pubmed_sampl 45%[========> ] 4.05M 6.28MB/s  pubmed_sample 95%[==================> ] 8.56M 9.96MB/s pubmed_sample_text_ 100%[===================>] 8.93M 10.4MB/s in 0.9s 

2021-04-29 20:31:08 (10.4 MB/s) - ‘pubmed_sample_text_small.csv’ saved [9363435/9363435]

In [0]:
import pyspark.sql.functions as F

pubMedDF = spark.read\
                .option("header", "true")\
                .csv("/pubmed_sample_text_small.csv")\
                
pubMedDF.show(truncate=50)

+--------------------------------------------------+
 text|
+--------------------------------------------------+
The human KCNJ9 (Kir 3.3, GIRK3) is a member of...|
BACKGROUND: At present, it is one of the most i...|
OBJECTIVE: To investigate the relationship betw...|
Combined EEG/fMRI recording has been used to lo...|
Kohlschutter syndrome is a rare neurodegenerati...|
Statistical analysis of neuroimages is commonly...|
The synthetic DOX-LNA conjugate was characteriz...|
Our objective was to compare three different me...|
We conducted a phase II study to assess the eff...|
"""Monomeric sarcosine oxidase (MSOX) is a flav...|
We presented the tachinid fly Exorista japonica...|
The literature dealing with the water conductin...|
A novel approach to synthesize chitosan-O-isopr...|
An HPLC-ESI-MS-MS method has been developed for...|
The localizing and lateralizing values of eye a...|
OBJECTIVE: To evaluate the effectiveness and ac...|
For the construction of new combinatorial libra...|
We report the results of a screen for genetic a...|
Intraparenchymal pericatheter cyst is rarely re...|
It is known that patients with Klinefelter's sy...|
+--------------------------------------------------+
only showing top 20 rows

In [0]:
result = model.transform(pubMedDF.limit(100))

In [0]:
result.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
 text| document| sentence| token| embeddings| ner| ner_chunk| assertion|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
The human KCNJ9 (...|[[document, 0, 95...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 48, 106,...|[[assertion, 48, ...|
BACKGROUND: At pr...|[[document, 0, 14...|[[document, 0, 19...|[[token, 0, 9, BA...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 67, 79, ...|[[assertion, 67, ...|
OBJECTIVE: To inv...|[[document, 0, 15...|[[document, 0, 30...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 188, 231...|[[assertion, 188,...|
Combined EEG/fMRI...|[[document, 0, 16...|[[document, 0, 16...|[[token, 0, 7, Co...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 26, C...|[[assertion, 0, 2...|
Kohlschutter synd...|[[document, 0, 25...|[[document, 0, 20...|[[token, 0, 11, K...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 20, K...|[[assertion, 0, 2...|
Statistical analy...|[[document, 0, 10...|[[document, 0, 34...|[[token, 0, 10, S...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 34, S...|[[assertion, 0, 3...|
The synthetic DOX...|[[document, 0, 57...|[[document, 0, 10...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 30, T...|[[assertion, 0, 3...|
Our objective was...|[[document, 0, 24...|[[document, 0, 19...|[[token, 0, 2, Ou...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 56, 80, ...|[[assertion, 56, ...|
We conducted a ph...|[[document, 0, 14...|[[document, 0, 24...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 28, ...|[[assertion, 13, ...|
"""Monomeric sarc...|[[document, 0, 14...|[[document, 0, 14...|[[token, 0, 2, ""...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 3, 36, M...|[[assertion, 3, 3...|
We presented the ...|[[document, 0, 12...|[[document, 0, 26...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 13, 46, ...|[[assertion, 13, ...|
The literature de...|[[document, 0, 16...|[[document, 0, 13...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 154, 166...|[[assertion, 154,...|
A novel approach ...|[[document, 0, 64...|[[document, 0, 97...|[[token, 0, 0, A,...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 99, 124,...|[[assertion, 99, ...|
An HPLC-ESI-MS-MS...|[[document, 0, 90...|[[document, 0, 24...|[[token, 0, 1, An...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 23, A...|[[assertion, 0, 2...|
The localizing an...|[[document, 0, 72...|[[document, 0, 12...|[[token, 0, 2, Th...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 50, 70, ...|[[assertion, 50, ...|
OBJECTIVE: To eva...|[[document, 0, 13...|[[document, 0, 43...|[[token, 0, 8, OB...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 154, 186...|[[assertion, 154,...|
For the construct...|[[document, 0, 32...|[[document, 0, 20...|[[token, 0, 2, Fo...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 24, 50, ...|[[assertion, 24, ...|
We report the res...|[[document, 0, 13...|[[document, 0, 38...|[[token, 0, 1, We...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 25, 32, ...|[[assertion, 25, ...|
Intraparenchymal ...|[[document, 0, 10...|[[document, 0, 53...|[[token, 0, 15, I...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 0, 33, I...|[[assertion, 0, 3...|
It is known that ...|[[document, 0, 34...|[[document, 0, 14...|[[token, 0, 1, It...|[[word_embeddings...|[[named_entity, 0...|[[chunk, 31, 52, ...|[[assertion, 31, ...|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
only showing top 20 rows

In [0]:
result.select('sentence.result').take(1)

Out[10]: [Row(result=['The human KCNJ9 (Kir 3.3, GIRK3) is a member of the G-protein-activated inwardly rectifying potassium (GIRK) channel family.', 'Here we describe the genomicorganization of the KCNJ9 locus on chromosome 1q21-23 as a candidate gene forType II diabetes mellitus in the Pima Indian population.', 'The gene spansapproximately 7.6 kb and contains one noncoding and two coding exons separated byapproximately 2.2 and approximately 2.6 kb introns, respectively.', 'We identified14 single nucleotide polymorphisms (SNPs), including one that predicts aVal366Ala substitution, and an 8 base-pair', '(bp) insertion/deletion.', 'Ourexpression studies revealed the presence of the transcript in various humantissues including pancreas, and two major insulin-responsive tissues: fat andskeletal muscle.', 'The characterization of the KCNJ9 gene should facilitate furtherstudies on the function of the KCNJ9 protein and allow evaluation of thepotential role of the locus in Type II diabetes.'])]

In [0]:
result.select(F.explode(F.arrays_zip('ner_chunk.result',  'ner_chunk.begin', 'ner_chunk.end', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']").alias("begin"),
        F.expr("cols['2']").alias("end"),
        F.expr("cols['3']['entity']").alias("ner_label"),
        F.expr("cols['3']['sentence']").alias("sent_id"),
        F.expr("cols['4']").alias("assertion"),).show(truncate=False)


+-----------------------------------------------------------+-----+---+---------+-------+-----------+
chunk |begin|end|ner_label|sent_id|assertion |
+-----------------------------------------------------------+-----+---+---------+-------+-----------+
the G-protein-activated inwardly rectifying potassium (GIRK|48 |106|TREATMENT|0 |conditional|
the genomicorganization |142 |164|TREATMENT|1 |present |
a candidate gene forType II diabetes mellitus |210 |254|PROBLEM |1 |present |
byapproximately |380 |394|TREATMENT|2 |present |
single nucleotide polymorphisms |464 |494|TREATMENT|3 |present |
aVal366Ala substitution |532 |554|TREATMENT|3 |present |
an 8 base-pair |561 |574|TREATMENT|3 |present |
insertion/deletion |581 |598|PROBLEM |4 |present |
Ourexpression studies |601 |621|TEST |5 |present |
the transcript in various humantissues |648 |685|PROBLEM |5 |present |
fat andskeletal muscle |749 |770|PROBLEM |5 |possible |
furtherstudies |830 |843|PROBLEM |6 |present |
the KCNJ9 protein |864 |880|TREATMENT|6 |present |
evaluation |892 |901|TEST |6 |possible |
Type II diabetes |940 |955|PROBLEM |6 |present |
the treatment |67 |79 |TREATMENT|0 |present |
breast cancer |84 |96 |PROBLEM |0 |present |
the standard therapy |109 |128|TREATMENT|0 |present |
anthracyclines |167 |180|TREATMENT|0 |present |
taxanes |186 |192|TREATMENT|0 |present |
+-----------------------------------------------------------+-----+---+---------+-------+-----------+
only showing top 20 rows

## Pretrained `assertion_dl_radiology` model

In [0]:
from sparknlp_jsl.annotator import *

documentAssembler = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

# Sentence Detector annotator, processes various sentences per line
sentenceDetector = SentenceDetectorDLModel\
    .pretrained("sentence_detector_dl_healthcare","en","clinical/models") \
    .setInputCols(["document"]) \
    .setOutputCol("sentence")

# Tokenizer splits words in a relevant format for NLP
tokenizer = Tokenizer()\
    .setInputCols(["sentence"])\
    .setOutputCol("token")

# Clinical word embeddings trained on PubMED dataset
word_embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
    .setInputCols(["sentence", "token"])\
    .setOutputCol("embeddings")

# NER model for radiology
radiology_ner = MedicalNerModel.pretrained("ner_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "token", "embeddings"]) \
    .setOutputCol("ner")

ner_converter = NerConverter() \
    .setInputCols(["sentence", "token", "ner"]) \
    .setOutputCol("ner_chunk")\
    .setWhiteList(["ImagingFindings"])

# Assertion model trained on radiology dataset
# coming from sparknlp_jsl.annotator !!

radiology_assertion = AssertionDLModel.pretrained("assertion_dl_radiology", "en", "clinical/models") \
    .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
    .setOutputCol("assertion")

nlpPipeline = Pipeline(stages=[
    documentAssembler, 
    sentenceDetector,
    tokenizer,
    word_embeddings,
    radiology_ner,
    ner_converter,
    radiology_assertion
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
radiologyAssertion_model = nlpPipeline.fit(empty_data)

sentence_detector_dl_healthcare download started this may take some time.
Approximate size to download 363.9 KB
[ | ][ / ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
ner_radiology download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]
assertion_dl_radiology download started this may take some time.
Approximate size to download 1.3 MB
[ | ][ / ][OK!]

In [0]:
# A sample text from a radiology report

text = """No right-sided pleural effusion or pneumothorax is definitively seen and there are mildly displaced fractures of the left lateral 8th and likely 9th ribs."""

In [0]:
data = spark.createDataFrame([[text]]).toDF("text")

In [0]:
result = radiologyAssertion_model.transform(data)

In [0]:
result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("chunk"),
        F.expr("cols['1']['entity']").alias("ner_label"),
        F.expr("cols['1']['sentence']").alias("sent_id"),
        F.expr("cols['2']").alias("assertion")).show(truncate=False)

+-------------------+---------------+-------+---------+
chunk |ner_label |sent_id|assertion|
+-------------------+---------------+-------+---------+
effusion |ImagingFindings|0 |Negative |
pneumothorax |ImagingFindings|0 |Negative |
displaced fractures|ImagingFindings|0 |Confirmed|
+-------------------+---------------+-------+---------+

## Writing a generic Assertion + NER function

In [0]:
from pyspark.sql.functions import monotonically_increasing_id


def get_base_pipeline (embeddings = 'embeddings_clinical'):

    documentAssembler = DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")

  # Sentence Detector annotator, processes various sentences per line

    sentenceDetector = SentenceDetector()\
        .setInputCols(["document"])\
        .setOutputCol("sentence")

  # Tokenizer splits words in a relevant format for NLP

    tokenizer = Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

  # Clinical word embeddings trained on PubMED dataset
    word_embeddings = WordEmbeddingsModel.pretrained(embeddings, "en", "clinical/models")\
        .setInputCols(["sentence", "token"])\
        .setOutputCol("embeddings")

    base_pipeline = Pipeline(stages=[
                        documentAssembler,
                        sentenceDetector,
                        tokenizer,
                        word_embeddings])

    return base_pipeline



def get_clinical_assertion (embeddings, spark_df, nrows = 100, model_name = 'ner_clinical'):

  # NER model trained on i2b2 (sampled from MIMIC) dataset
    loaded_ner_model = MedicalNerModel.pretrained(model_name, "en", "clinical/models") \
        .setInputCols(["sentence", "token", "embeddings"]) \
        .setOutputCol("ner")

    ner_converter = NerConverter() \
        .setInputCols(["sentence", "token", "ner"]) \
        .setOutputCol("ner_chunk")

  # Assertion model trained on i2b2 (sampled from MIMIC) dataset
  # coming from sparknlp_jsl.annotator !!
    clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
        .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
        .setOutputCol("assertion")
      

    base_model = get_base_pipeline (embeddings)

    nlpPipeline = Pipeline(stages=[
        base_model,
        loaded_ner_model,
        ner_converter,
        clinical_assertion])

    empty_data = spark.createDataFrame([[""]]).toDF("text")

    model = nlpPipeline.fit(empty_data)

    result = model.transform(spark_df.limit(nrows))

    result = result.withColumn("id", monotonically_increasing_id())

    result_df = result.select(F.explode(F.arrays_zip('ner_chunk.result', 'ner_chunk.metadata', 'assertion.result')).alias("cols")) \
      .select(F.expr("cols['0']").alias("chunk"),
          F.expr("cols['1']['entity']").alias("ner_label"),
          F.expr("cols['2']").alias("assertion"))\
           .filter("ner_label!='O'")

    return result_df

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_clinical'

nrows = 1000

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+--------------------+---------+-----------+
 chunk|ner_label| assertion|
+--------------------+---------+-----------+
the G-protein-act...|TREATMENT|conditional|
the genomicorgani...|TREATMENT| present|
a candidate gene ...| PROBLEM| present|
 byapproximately|TREATMENT| present|
single nucleotide...|TREATMENT| present|
aVal366Ala substi...|TREATMENT| present|
 an 8 base-pair|TREATMENT| present|
 insertion/deletion| PROBLEM| present|
Ourexpression stu...| TEST| present|
the transcript in...| PROBLEM| present|
fat andskeletal m...| PROBLEM| possible|
 furtherstudies| PROBLEM| present|
 the KCNJ9 protein|TREATMENT| present|
 evaluation| TEST| possible|
 Type II diabetes| PROBLEM| present|
 the treatment|TREATMENT| present|
 breast cancer| PROBLEM| present|
the standard therapy|TREATMENT| present|
 anthracyclines|TREATMENT| present|
 taxanes|TREATMENT| present|
+--------------------+---------+-----------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show()

ner_posology download started this may take some time.
Approximate size to download 13.8 MB
[ | ][ / ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+---------+---------+
 chunk|ner_label|assertion|
+----------------+---------+---------+
 anthracyclines| DRUG| present|
 taxanes| DRUG| present|
 vinorelbine| DRUG| present|
 vinorelbine| DRUG| present|
 anthracyclines| DRUG| present|
 taxanes| DRUG| present|
 Vinorelbinewas| DRUG| absent|
 25 mg/m(2| STRENGTH| present|
 intravenously| ROUTE| absent|
 on days|FREQUENCY| present|
tumorprogression| DRUG| present|
 thatvinorelbine| DRUG| present|
 anthracyclines| DRUG| present|
 taxanes| DRUG| present|
 tetrazolium dye| DRUG| present|
 DOX| DRUG| absent|
 trandolapril| DRUG| possible|
 losartan| DRUG| present|
 3-week| DURATION| absent|
 placebo| DRUG| present|
+----------------+---------+---------+
only showing top 20 rows

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_posology_greedy'

entry_data = spark.createDataFrame([["The patient did not take a capsule of Advil"]]).toDF("text")

ner_df = get_clinical_assertion (embeddings, entry_data, nrows, model_name)

ner_df.show()

ner_posology_greedy download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------+---------+---------+
 chunk|ner_label|assertion|
+----------------+---------+---------+
capsule of Advil| DRUG| absent|
+----------------+---------+---------+

In [0]:
import pandas as pd

def get_clinical_assertion_light (light_model, text):

  light_result = light_model.fullAnnotate(text)[0]

  chunks=[]
  entities=[]
  status=[]

  for n,m in zip(light_result['ner_chunk'],light_result['assertion']):
      
      chunks.append(n.result)
      entities.append(n.metadata['entity']) 
      status.append(m.result)
          
  df = pd.DataFrame({'chunks':chunks, 'entities':entities, 'assertion':status})

  return df

In [0]:
clinical_text = """
Patient with severe fever and sore throat. 
He shows no stomach pain and he maintained on an epidural and PCA for pain control.
He also became short of breath with climbing a flight of stairs.
After CT, lung tumor located at the right lower lobe. Father with Alzheimer.
"""

light_model = LightPipeline(model)

get_clinical_assertion_light (light_model, clinical_text)


Out[33]:

,chunks,entities,assertion
0,severe fever,PROBLEM,present
1,sore throat,PROBLEM,present
2,stomach pain,PROBLEM,absent
3,an epidural,TREATMENT,present
4,PCA,TREATMENT,present
5,pain control,PROBLEM,present
6,short of breath,PROBLEM,conditional
7,CT,TEST,present
8,lung tumor,PROBLEM,present
9,Alzheimer,PROBLEM,associated_with_someone_else


## Assertion with BioNLP (Cancer Genetics) NER

In [0]:
embeddings = 'embeddings_clinical'

model_name = 'ner_bionlp'

nrows = 100

ner_df = get_clinical_assertion (embeddings, pubMedDF, nrows, model_name)

ner_df.show(truncate = False)

ner_bionlp download started this may take some time.
Approximate size to download 13.9 MB
[ | ][ / ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][OK!]
+----------------------+--------------------+-----------+
chunk |ner_label |assertion |
+----------------------+--------------------+-----------+
human |Organism |present |
Kir 3.3 |Gene_or_gene_product|present |
GIRK3 |Gene_or_gene_product|present |
potassium |Simple_chemical |conditional|
GIRK |Gene_or_gene_product|conditional|
chromosome 1q21-23 |Cellular_component |present |
pancreas |Organ |present |
tissues |Tissue |possible |
fat andskeletal muscle|Tissue |possible |
KCNJ9 |Gene_or_gene_product|present |
Type II |Gene_or_gene_product|present |
breast cancer |Cancer |present |
patients |Organism |present |
anthracyclines |Simple_chemical |present |
taxanes |Simple_chemical |present |
vinorelbine |Simple_chemical |present |
patients |Organism |present |
breast |Cancer |present |
vinorelbine inpatients|Simple_chemical |present |
anthracyclines |Simple_chemical |present |
+----------------------+--------------------+-----------+
only showing top 20 rows

# Assertion Filterer
AssertionFilterer will allow you to filter out the named entities by the list of acceptable assertion statuses. This annotator would be quite handy if you want to set a white list for the acceptable assertion statuses like present or conditional; and do not want absent conditions get out of your pipeline.

In [0]:
clinical_ner = MedicalNerModel.pretrained("ner_clinical", "en", "clinical/models") \
      .setInputCols(["sentence", "token", "embeddings"]) \
      .setOutputCol("ner")

ner_converter = NerConverter() \
      .setInputCols(["sentence", "token", "ner"]) \
      .setOutputCol("ner_chunk")

clinical_assertion = AssertionDLModel.pretrained("assertion_dl", "en", "clinical/models") \
      .setInputCols(["sentence", "ner_chunk", "embeddings"]) \
      .setOutputCol("assertion")

assertion_filterer = AssertionFilterer()\
      .setInputCols("sentence","ner_chunk","assertion")\
      .setOutputCol("assertion_filtered")\
      .setWhiteList(["present"])

nlpPipeline = Pipeline(stages=[
      documentAssembler, 
      sentenceDetector,
      tokenizer,
      word_embeddings,
      clinical_ner,
      ner_converter,
      clinical_assertion,
      assertion_filterer
    ])

empty_data = spark.createDataFrame([[""]]).toDF("text")
assertionFilter_model = nlpPipeline.fit(empty_data)

ner_clinical download started this may take some time.
Approximate size to download 13.9 MB
[ | ][OK!]
assertion_dl download started this may take some time.
Approximate size to download 1.3 MB
[ | ][OK!]

In [0]:
text = 'Patient has a headache for the last 2 weeks, needs to get a head CT, and appears anxious when she walks fast. Alopecia noted. She denies pain.'

light_model = LightPipeline(assertionFilter_model)
light_result = light_model.annotate(text)

light_result.keys()

Out[36]: dict_keys(['assertion_filtered', 'document', 'ner_chunk', 'assertion', 'token', 'ner', 'embeddings', 'sentence'])

In [0]:
list(zip(light_result['ner_chunk'], light_result['assertion']))

Out[37]: [('a headache', 'present'),
 ('a head CT', 'present'),
 ('anxious', 'present'),
 ('Alopecia', 'present'),
 ('pain', 'absent')]

In [0]:
assertion_filterer.getWhiteList()

Out[38]: ['present']

In [0]:
light_result['assertion_filtered']

Out[39]: ['a headache', 'a head CT', 'anxious', 'Alopecia']

# Train a custom Assertion Model

In [0]:
!wget https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/ner/i2b2_assertion_sample.csv

--2021-04-30 06:45:56-- https://s3.amazonaws.com/auxdata.johnsnowlabs.com/public/resources/en/ner/i2b2_assertion_sample.csv
Resolving s3.amazonaws.com (s3.amazonaws.com)... 52.216.92.93
Connecting to s3.amazonaws.com (s3.amazonaws.com)|52.216.92.93|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 141585 (138K) [text/csv]
Saving to: ‘i2b2_assertion_sample.csv’

 i2b2_asse 0%[ ] 0 --.-KB/s  i2b2_asser 79%[==============> ] 109.60K 475KB/s i2b2_assertion_samp 100%[===================>] 138.27K 599KB/s in 0.2s 

2021-04-30 06:45:56 (599 KB/s) - ‘i2b2_assertion_sample.csv’ saved [141585/141585]

In [0]:
%sh ls -la

total 168
drwxr-xr-x 1 root root 4096 Apr 30 06:45 .
drwxr-xr-x 1 root root 4096 Apr 30 06:43 ..
drwxr-xr-x 2 root root 4096 Jan 1 1970 conf
drwxr-xr-x 3 root root 4096 Apr 30 06:43 eventlogs
drwxr-xr-x 2 root root 4096 Apr 30 06:45 ganglia
-rw-r--r-- 1 root root 141585 Apr 14 2020 i2b2_assertion_sample.csv
drwxr-xr-x 2 root root 4096 Apr 30 06:43 logs

In [0]:
assertion_df = spark.read.option("header", True).option("inferSchema", "True").csv("/i2b2_assertion_sample.csv")


In [0]:
assertion_df.show(3, truncate=100)

+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
 text| target| label|start|end|
+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
Due to dye load required to assess the coronary disease were not visualized .|the coronary disease|present| 7| 9|
 She has no history of liver disease , hepatitis .| liver disease| absent| 5| 6|
 1. Undesired fertility .| undesired fertility|present| 1| 2|
+-----------------------------------------------------------------------------+--------------------+-------+-----+---+
only showing top 3 rows

In [0]:
(training_data, test_data) = assertion_df.randomSplit([0.8, 0.2], seed = 100)
print("Training Dataset Count: " + str(training_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Training Dataset Count: 814
Test Dataset Count: 186

In [0]:
training_data.groupBy('label').count().orderBy('count', ascending=False).show(truncate=False)


+----------------------------+-----+
label |count|
+----------------------------+-----+
present |546 |
absent |182 |
hypothetical |41 |
possible |34 |
associated_with_someone_else|6 |
conditional |5 |
+----------------------------+-----+

In [0]:
document = DocumentAssembler()\
    .setInputCol("text")\
    .setOutputCol("document")

chunk = Doc2Chunk()\
    .setInputCols("document")\
    .setOutputCol("chunk")\
    .setChunkCol("target")\
    .setStartCol("start")\
    .setStartColByTokenIndex(True)\
    .setFailOnMissing(False)\
    .setLowerCase(True)

token = Tokenizer()\
    .setInputCols(['document'])\
    .setOutputCol('token')

embeddings = WordEmbeddingsModel.pretrained("embeddings_clinical", "en", "clinical/models")\
      .setInputCols(["document", "token"])\
      .setOutputCol("embeddings")


embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][ | ][ / ][ — ][ \ ][OK!]

In [0]:
!mkdir training_logs
!mkdir tf_graphs
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/nerdl_graph/blstm_34_32_30_200_6.pb -P /databricks/driver/tf_graphs

In [0]:
assertionStatus = AssertionDLApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setBatchSize(128)\
    .setDropout(0.1)\
    .setLearningRate(0.001)\
    .setEpochs(50)\
    .setValidationSplit(0.2)\
    .setStartCol("start")\
    .setEndCol("end")\
    .setMaxSentLen(250)\
    .setEnableOutputLogs(True)\
    .setOutputLogsPath('training_logs/')\
    .setGraphFolder('file:/databricks/driver/tf_graphs')

# set this path to graphs if you have different nuber of classes other than 6. (use 2.1 Asertion Model Graph generation notebook)

In [0]:
from sparknlp_jsl.training import tf_graph
import sys

sys.path.append('/databricks/driver/tf_graph')


tf_graph.print_model_params("assertion_dl")
feat_size = 200
n_classes = 6
tf_graph.build("assertion_dl",build_params={"n_classes": n_classes}, model_location="dbfs:/databricks/driver/tf_graphs", model_filename="blstm_34_32_30_{}_{}.pb".format(feat_size, n_classes))

In [0]:
'''
assertionStatus = AssertionLogRegApproach()\
    .setLabelCol("label")\
    .setInputCols("document", "chunk", "embeddings")\
    .setOutputCol("assertion")\
    .setMaxIter(100) # default: 26
'''

Out[26]: '\nassertionStatus = AssertionLogRegApproach() .setLabelCol("label") .setInputCols("document", "chunk", "embeddings") .setOutputCol("assertion") .setMaxIter(100) # default: 26\n'

In [0]:
clinical_assertion_pipeline = Pipeline(
    stages = [
    document,
    chunk,
    token,
    embeddings,
    assertionStatus])

In [0]:
assertion_model = clinical_assertion_pipeline.fit(training_data)

In [0]:
%sh cd training_logs && ls -lt

total 36
-rw-r--r-- 1 root root 33486 Apr 30 06:57 AssertionDLApproach_cd394b94a15c.log

In [0]:
%sh tail -n 26 training_logs/AssertionDLApproach_cd394b94a15c.log

present 107 19 6 0.8492063 0.94690263 0.8953975
conditional 0 0 2 0.0 0.0 0.0
associated_with_someone_else 0 0 2 0.0 0.0 0.0
hypothetical 1 2 1 0.33333334 0.5 0.4
possible 0 0 2 0.0 0.0 0.0
tp: 137 fp: 25 fn: 25 labels: 6
Macro-average prec: 0.3435546, rec: 0.3590366, f1: 0.351125
Micro-average prec: 0.845679, rec: 0.845679, f1: 0.845679


Epoch: 49 started, learning rate: 8.099466E-5, dataset size: 652
Done, 4.719263598 loss: 3.3174746, batches: 6
Quality on validation dataset (20.0%), validation examples = 162
time to finish evaluation: 0.77s
label tp fp fn prec rec f1
absent 29 4 12 0.8787879 0.70731705 0.78378385
present 107 19 6 0.8492063 0.94690263 0.8953975
conditional 0 0 2 0.0 0.0 0.0
associated_with_someone_else 0 0 2 0.0 0.0 0.0
hypothetical 1 2 1 0.33333334 0.5 0.4
possible 0 0 2 0.0 0.0 0.0
tp: 137 fp: 25 fn: 25 labels: 6
Macro-average prec: 0.3435546, rec: 0.3590366, f1: 0.351125
Micro-average prec: 0.845679, rec: 0.845679, f1: 0.845679

In [0]:
preds = assertion_model.transform(test_data).select('label','assertion.result')

preds.show()

+------------+---------+
 label| result|
+------------+---------+
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
hypothetical|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
 present|[present]|
+------------+---------+
only showing top 20 rows

In [0]:
preds_df = preds.toPandas()


In [0]:
preds_df['result'] = preds_df['result'].apply(lambda x : x[0])


In [0]:
# We are going to use sklearn to evalute the results on test dataset
from sklearn.metrics import classification_report

print (classification_report(preds_df['result'], preds_df['label']))

/databricks/python/lib/python3.7/site-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
 _warn_prf(average, modifier, msg_start, len(result))
 precision recall f1-score support

 absent 0.61 0.76 0.67 37
associated_with_someone_else 0.00 0.00 0.00 0
 conditional 0.00 0.00 0.00 0
 hypothetical 0.44 0.44 0.44 9
 possible 0.00 0.00 0.00 0
 present 0.91 0.76 0.83 140

 accuracy 0.75 186
 macro avg 0.33 0.33 0.32 186
 weighted avg 0.83 0.75 0.78 186

In [0]:
assertion_model.stages[-1].write().overwrite().save('/databricks/driver/models/assertion_model_0429')